<a href="https://colab.research.google.com/github/VL98/VLehner-Bachelor-Thesis/blob/main/Basic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Model

### Loading libraries
Initializing variables

In [ ]:

install.packages(c("nloptr", "rootSolve", "pracma", "NlcOptim","parallel", "ragtop"))

library(ragtop)
library(pracma)
library(nloptr)
library(NlcOptim)
library(rootSolve)
library(stats)
library(parallel)
option = read.csv("option15clean.csv")
v = unique(option$date)
n = 10^5
x_2 = read.csv("x2.csv")$x
x = x_2[1:5]

In [ ]:
option = read.csv("option15clean.csv")
v = unique(option$date)
n = 10^5
x_2 = read.csv("x2.csv")$x
x = x_2[1:5]
date = v[1]

### Function definitions 

Part I

In [ ]:
HNGP = function (TypeFlag = c("c", "p"), model, S, X, Time.inDays, 
          r.daily) {


  .fstarHN = function (phi, const, model, S, X, Time.inDays, r.daily){
    
  lambda = -1/2
  omega = model$omega
  alpha = model$alpha
  gamma = model$gamma 
  beta = model$beta
  sigma2 = model$sigma2
  cphi0 = phi * complex(real = 0, imaginary = 1)
  cphi = cphi0 + const
  a = cphi * r.daily
  b = lambda * cphi + cphi * cphi/2
  for (i in 2:Time.inDays) {
    a = a + cphi * r.daily + b * omega - log(1 - 2 * alpha * 
                                               b)/2
    b = cphi * (lambda + gamma) - gamma^2/2 + beta * b + 
      0.5 * (cphi - gamma)^2/(1 - 2 * alpha * b)
  }
  f = Re(exp(-cphi0 * log(X) + cphi * log(S) + a + b * sigma2)/cphi0)/pi
  f
}
  TypeFlag = TypeFlag[1]
  call1 = integrate(.fstarHN, 0, Inf, const = 1, model = model, subdivisions = 1000L,  
                    S = S, X = X, Time.inDays = Time.inDays, r.daily = r.daily)
  if (is.null(call1$value)) 
    call1$value = call1$integral
  call2 = integrate(.fstarHN, 0, Inf, const = 0, model = model, subdivisions = 1000L,  
                    S = S, X = X, Time.inDays = Time.inDays, r.daily = r.daily)
  if (is.null(call2$value)) 
    call2$value = call2$integral
  call.price = S/2 + exp(-r.daily * Time.inDays) * call1$value - 
    X * exp(-r.daily * Time.inDays) * (1/2 + call2$value)
  price = NA
  if (TypeFlag == "c") 
    price = call.price
  if (TypeFlag == "p") 
    price = call.price + X * exp(-r.daily * Time.inDays) - 
    S
  option = list(price = price, call = match.call())
  class(option) = "option"
  option
}

###Function defintion

Part II

In [ ]:
scaler = function(x){
  y = x
  y[1] = y[1]*1e07
  y[2] = y[2]*1e06
  y[3] = y[3] * 10
  y[4] = y[4]/100
  y[5] = y[5]* 10^3
  y
}

rescaler = function(x){
  y = x
  y[1] = y[1]/1e07
  y[2] = y[2]/1e06
  y[3] = y[3] / 10
  y[4] = y[4]*100
  y[5] = y[5]/ 10^3
  
  y
}

fN1 = function(x){
 x = rescaler(x)
  A = function(y, x) {
    c = try(HNGP(TypeFlag = "c", model = list(omega = x[1], alpha = x[2], beta = x[3],
                                          gamma = x[4], sigma2 = x[5]), S = y[2],
             X = y[1], Time.inDays = y[3], r.daily =  y[4]/(100*252) )$price)
   if(class(c) == "try-error"| is.na(c)){
     ##return(sample(c(6,5), size = 1))
     return(5)
   }
   else{
    iv = try(implied_volatilities(c, 1, y[2], y[1], y[4]/100, y[3]/252))
    ifelse(class(iv) == "try-error"| is.na(iv), 6, iv)
   }
  }

  
t_size =  option$date ==  date
  CP = apply(option[t_size, c("strike_price", "SPOT", "TTM", "interest") ],1, A, x = x)
  
  
  
  mean((CP - option$own_impvol[t_size])^2)
}


FN1_final= function(x){
  a = try(fN1(x))
  ifelse(class(a)== "try-error", 5, a)
}


hin3 = function(x) {
  x = rescaler(x)
  h = numeric(1)
  h[1] = x[2]*(x[4])^2  + x[3] - 1 + 1e-15
  return(h)
}


### Function execution

In [ ]:
boundslow = c(1e-20, 0, 0, -Inf, 1e-20) 
boundsup  = c(Inf, Inf, Inf, Inf, 1)

start = Sys.time()
Model = fmincon(scaler(x), FN1_final, lb = scaler(boundslow), ub = scaler(boundsup), hin =  hin3, tol = 1e-07 )
Model$par = rescaler(Model$par)
dur = Sys.time() - start


In [ ]:
#hin3(scaler(x))
i = 7
HNGP(TypeFlag = "c", model = list(omega = x[1], alpha = x[2], beta = x[3],
                                          gamma = x[4], sigma2 = x[5]), S = option$SPOT[i],
             X = option$strike[i], Time.inDays = option$TTM[i], r.daily =  option$interest[i]/(100*252) )$price

option$price[i]

In [ ]:
funcons2 = function(x) {
  x = rescaler(x)
  h = numeric(9)
  h[1] = -x[2]*(x[4] )^2  + -x[3] + 1 - 1e-20
  h[2] = x[2]
  h[3] = x[3] 
  h[4] = x[1]  - 1e-20
  h[5] = x[5] - 1e-20
  
  return(h)
}



start = Sys.time()
Model = cobyla(x0 = scaler(x), fn  = FN1_final,  hin = funcons2, nl.info =  T,
                control = list(xtol_rel = 1e-8, maxeval = 1000))
dur = Sys.time() - start


In [ ]:
FN1_final(x)
scaler(x)

In [ ]:
fN1R = function(x){
 
  A = function(y, x) {
    c = try(HNGP(TypeFlag = "c", model = list(omega = x[1], alpha = x[2], beta = x[3],
                                          gamma = x[4], sigma2 = x[5]), S = y[2],
             X = y[1], Time.inDays = y[3], r.daily =  y[4]/(100*252) )$price)
   if(class(c) == "try-error"| is.na(c)){
     ##return(sample(c(6,5), size = 1))
     return(5)
   }
   else{
    c
   }
  }

  
t_size =  option$date ==  date
  CP = apply(option[t_size, c("strike_price", "SPOT", "TTM", "interest") ],1, A, x = x)
  
  
  
  mean((CP - option$price[t_size])^2)
}


In [ ]:
fN1R(Model$par)

x = Model$par